
## R에서 간단한 처리 이후 이쪽으로 분석 진행. 
### 처리내역 :
#### Outlier 및 NA observation 처리 : 
* 4분 이하, 4000분 이상 데이터 삭제
* confidence NA 변수 5000여개 삭제
* Salary NA -> I do Not want to disclose로 합침. 

#### 변수 One Hot Encoding : 
* 기존에 One Hot Encoding 되어있는 변수들은 0, 1 로 재코딩
* class를 보유하고 있는 단일변수들은 one hot encoding (get_dummies) 하여 여러 변수로 나누어줌. (ex - Role, Industry, gender, ...)
* Age : 이번에는 One hot encoding (즉, 7~8개의 변수가 되도록) 
* 척도형 변수를 갖는 단일 변수 => 이번에는 One Hot Encoding 진행. 
* Country = 5개 대륙으로 나눔
* Role(직업) : 총 7여개로 분류 - Business Analyst,	Data Analyst,	Data Engineer/DBA,	Data Scientist,	Software Engineer,	Statistician/Research Scientist,	Student
* Student : Role과 industry에 Student로 기재된 것 동일하게 바꿔줌 

#### 삭제한 변수 내역 : 
* R에서 삭제한 변수 (Q33, Q39, Q17, Q18, Q20, Q22, Q32, Q37, Q12 Multiple choice) 
* 총 Observation이 전체 observation의 5% 미만일 경우 삭제. (column 별로 다 더해서 /shape[0]으로 나눈 뒤 5% 미만의 변수들 삭제함. 


#### Target Variabel Salary 변형 : 
* 총 19개의 multiclass를 가지고 있어, multiclassification 하게 되면 성능이 너무 낮아지는 문제가 발생하게 됨. 
* 상위 20%에게 1, 하위 80%에게 0 label, 마지막으로 I do not want to disclose = 2로 하는 3 class 문제로 변형. 




In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
multiple = pd.read_csv("/content/drive/My Drive/multiple.csv", low_memory = False)

In [ ]:
qschema = pd.read_csv("/content/drive/My Drive/SurveySchema.csv")

In [ ]:
multiple.head()

,duration,gender,Age,country,degree,undergraduate_major,Role,industry,Experience,value,Q10,Q11_Part_1,Q11_Part_2,Q11_Part_3,Q11_Part_4,Q11_Part_5,Q11_Part_6,Q11_Part_7,Q12_Part_1_TEXT,Q12_Part_2_TEXT,Q12_Part_3_TEXT,Q12_Part_4_TEXT,Q12_Part_5_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,Q13_Part_6,Q13_Part_7,Q13_Part_8,Q13_Part_9,Q13_Part_10,Q13_Part_11,Q13_Part_12,Q13_Part_13,Q13_Part_14,Q13_Part_15,Q14_Part_1,Q14_Part_2,...,Q45_Part_5,Q45_Part_6,Q46,Q47_Part_1,Q47_Part_2,Q47_Part_3,Q47_Part_4,Q47_Part_5,Q47_Part_6,Q47_Part_7,Q47_Part_8,Q47_Part_9,Q47_Part_10,Q47_Part_11,Q47_Part_12,Q47_Part_13,Q47_Part_14,Q47_Part_15,Q47_Part_16,Q48,Q49_Part_1,Q49_Part_2,Q49_Part_3,Q49_Part_4,Q49_Part_5,Q49_Part_6,Q49_Part_7,Q49_Part_8,Q49_Part_9,Q49_Part_10,Q49_Part_11,Q49_Part_12,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8
0,718,Female,30-34,NorthAmerica,Master’s degree,"Computer science (software engineering, etc.)",Student,Student,0-1,"0-10,000",I do not know,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,MATLAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10-20,NaN,Examine feature correlations,Examine feature importances,NaN,NaN,NaN,NaN,Plot predicted vs. actual results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I am confident that I can explain the outputs ...,NaN,NaN,NaN,NaN,NaN,NaN,Make sure the code is human-readable,Define all random seeds,NaN,Include a text file describing all dependencies,NaN,NaN,NaN,Too time-consuming,NaN,NaN,NaN,NaN,NaN,NaN
1,731,Male,22-24,Asia,Master’s degree,Mathematics or statistics,Student,Student,0-1,"0-10,000",I do not know,NaN,NaN,NaN,NaN,NaN,NaN,Other,-1,1,-1,-1,-1,NaN,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20-30,NaN,NaN,NaN,NaN,Create partial dependence plots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I am confident that I can understand and expla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Define relative rather than absolute file paths,NaN,NaN,NaN,NaN,Too time-consuming,NaN,NaN,Not enough incentives to share my work,NaN,NaN,NaN
2,1142,Male,25-29,SouthAmerica,Bachelor’s degree,Physics or astronomy,Data Scientist,Computers/Technology,0-1,I do not wish to disclose my approximate yearl...,We are exploring ML methods (and may one day p...,Analyze and understand data to influence produ...,NaN,NaN,Build prototypes to explore applying machine l...,Do research that advances the state of the art...,NaN,NaN,-1,-1,-1,2,-1,Jupyter/IPython,NaN,NaN,NaN,NaN,Atom,NaN,NaN,NaN,NaN,Vim,NaN,NaN,NaN,NaN,Kaggle Kernels,Google Colab,...,When first exploring a new ML model or dataset,NaN,0,NaN,Examine feature correlations,Examine feature importances,NaN,NaN,NaN,NaN,Plot predicted vs. actual results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I view ML models as ""black boxes"" but I am con...",Share code on Github or a similar code-sharing...,Share both data and code on Github or a simila...,"Share data, code, and environment using a host...",NaN,"Share code, data, and environment using virtua...",Make sure the code is well documented,Make sure the code is human-readable,NaN,NaN,Include a text file describing all dependencies,NaN,NaN,NaN,NaN,NaN,Afraid that others will use my work without gi...,NaN,I had never considered making my work easier f...,NaN,NaN
3,751,Male,30-34,Europe,Master’s degree,Engineering (non-computer focused),Software Engineer,Online Service/Internet-based Services,3-4,"20-30,000","We have well established ML methods (i.e., mod...",NaN,Build and/or run a machine learning service th...,Build and/or run the data infrastructure that ...,NaN,NaN,NaN,NaN,-1,-1,-1,4,-1,Jupyter/IPython,NaN,PyCharm,Visual Studio Code,NaN,NaN,NaN,Visual Studio,NaN,Sublime Text,NaN,IntelliJ,NaN,NaN,NaN,Kaggle Kernels,NaN,...,When first exploring a new ML model or dataset,NaN,50-60,NaN,Examine feature correlations,NaN,NaN,NaN,Dimensionality reduction techniques,NaN,Plot predicted vs. actual r

In [ ]:
from pandas.api.types import CategoricalDtype

In [ ]:
multiple.dtypes

duration       int64
gender        object
Age           object
country       object
degree        object
               ...  
Q50_Part_4    object
Q50_Part_5    object
Q50_Part_6    object
Q50_Part_7    object
Q50_Part_8    object
Length: 346, dtype: object

In [ ]:
multiple.columns

Index(['duration', 'gender', 'Age', 'country', 'degree', 'undergraduate_major',
       'Role', 'industry', 'Experience', 'value',
       ...
       'Q49_Part_11', 'Q49_Part_12', 'Q50_Part_1', 'Q50_Part_2', 'Q50_Part_3',
       'Q50_Part_4', 'Q50_Part_5', 'Q50_Part_6', 'Q50_Part_7', 'Q50_Part_8'],
      dtype='object', length=346)

In [ ]:
multiple.iloc[:,319:321]

,Q47_Part_11,Q47_Part_12
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,ELI5 functions
...,...,...
14087,NaN,NaN
14088,NaN,NaN
14089,NaN,NaN
14090,NaN,NaN


In [ ]:
question = multiple.columns

In [ ]:
question[100:300]

Index(['Q21_Part_8', 'Q21_Part_9', 'Q21_Part_10', 'Q21_Part_11', 'Q21_Part_12',
       'Q21_Part_13', 'Q23', 'Q24', 'Q25', 'confidence',
       ...
       'Q42_Part_1', 'Q42_Part_2', 'Q42_Part_3', 'Q42_Part_4', 'Q42_Part_5',
       'Q43', 'Q44_Part_1', 'Q44_Part_2', 'Q44_Part_3', 'Q44_Part_4'],
      dtype='object', length=200)

In [ ]:
multiple.dtypes[10:20]

Q10                object
Q11_Part_1         object
Q11_Part_2         object
Q11_Part_3         object
Q11_Part_4         object
Q11_Part_5         object
Q11_Part_6         object
Q11_Part_7         object
Q12_Part_1_TEXT     int64
Q12_Part_2_TEXT     int64
dtype: object

## One Hot Encoding : 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
sns.set_style('dark')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


* Role : 

In [ ]:
len(multiple.Role[multiple.Role.isna()])

0

In [ ]:
roledummy = pd.get_dummies(multiple.Role)

In [ ]:
roledummy.head()

,Business Analyst,Data Analyst,Data Engineer/DBA,Data Scientist,Software Engineer,Statistician/Research Scientist,Student
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1


* Q1 - Q10 - gender

In [ ]:
multiple.gender.unique()

array(['Female', 'Male', 'Prefer not to say', 'Prefer to self-describe'],
      dtype=object)

In [ ]:
sex = pd.get_dummies(multiple.gender).iloc[:,:2]
sex.columns=list(map(lambda x:'1_'+x,sex.columns))

In [ ]:
sex[1:10]

,1_Female,1_Male
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1
8,1,0
9,0,1


In [ ]:
ndata = multiple.iloc[:,0]
ndata.head()

0     718
1     731
2    1142
3     751
4    2028
Name: duration, dtype: int64

In [ ]:
ndata = pd.concat([ndata,sex],axis=1)
ndata.head()

,duration,1_Female,1_Male
0,718,1,0
1,731,0,1
2,1142,0,1
3,751,0,1
4,2028,0,1


* Q1 - Q10 : 나이

In [ ]:
multiple.Age.unique()

array(['30-34', '22-24', '25-29', '40-44', '55-59', '18-21', '35-39',
       '45-49', '60-69', '50-54', '80+', '70-79'], dtype=object)

In [ ]:
q2=pd.get_dummies(multiple.Age)
q2.columns=list(map(lambda x: '2_'+x,q2.columns))
ndata = pd.concat([ndata,q2],axis=1)

country - one hot encoding 

In [ ]:
multiple.country.unique()

array(['NorthAmerica', 'Asia', 'SouthAmerica', 'Europe', 'Africa',
       'Oceania', nan], dtype=object)

In [ ]:
q3=pd.get_dummies(multiple.country)

In [ ]:
q3.columns=list(map(lambda x: '3_'+x,q3.columns))
ndata = pd.concat([ndata,q3],axis=1)

In [ ]:
ndata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14092 entries, 0 to 14091
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   duration        14092 non-null  int64
 1   1_Female        14092 non-null  uint8
 2   1_Male          14092 non-null  uint8
 3   2_18-21         14092 non-null  uint8
 4   2_22-24         14092 non-null  uint8
 5   2_25-29         14092 non-null  uint8
 6   2_30-34         14092 non-null  uint8
 7   2_35-39         14092 non-null  uint8
 8   2_40-44         14092 non-null  uint8
 9   2_45-49         14092 non-null  uint8
 10  2_50-54         14092 non-null  uint8
 11  2_55-59         14092 non-null  uint8
 12  2_60-69         14092 non-null  uint8
 13  2_70-79         14092 non-null  uint8
 14  2_80+           14092 non-null  uint8
 15  3_Africa        14092 non-null  uint8
 16  3_Asia          14092 non-null  uint8
 17  3_Europe        14092 non-null  uint8
 18  3_NorthAmerica  14092 non-

Degree - 교육수준 one hot encoding! 

In [ ]:
multiple.degree.unique()

array(['Master’s degree', 'Bachelor’s degree', 'Doctoral degree',
       'Some college/university study without earning a bachelor’s degree',
       'Professional degree', 'No formal education past high school'],
      dtype=object)

In [ ]:
q4 = pd.get_dummies(multiple.degree)
q4.columns=list(map(lambda x: '4_'+x,q4.columns))
ndata = pd.concat([ndata,q4],axis=1)

* Role 추가 

In [ ]:
ndata = pd.concat([ndata, roledummy], axis=1)

Major in undergraduated : 

In [ ]:
multiple.undergraduate_major.unique()

array(['Computer science (software engineering, etc.)',
       'Mathematics or statistics', 'Physics or astronomy',
       'Engineering (non-computer focused)',
       'Information technology, networking, or system administration',
       'A business discipline (accounting, economics, finance, etc.)',
       'Environmental science or geology', 'I never declared a major',
       'Medical or life sciences (biology, chemistry, medicine, etc.)',
       'Other',
       'Social sciences (anthropology, psychology, sociology, etc.)',
       'Humanities (history, literature, philosophy, etc.)', nan,
       'Fine arts or performing arts'], dtype=object)

In [ ]:
q5 = pd.get_dummies(multiple.undergraduate_major)
q5.columns=list(map(lambda x: '5_'+x,q5.columns))
ndata = pd.concat([ndata,q5],axis=1)

* Industry : 산업군! 

In [ ]:
q7 = pd.get_dummies(multiple.industry)
q7.columns=list(map(lambda x: '7_'+x,q7.columns))
ndata = pd.concat([ndata,q7],axis=1)

* Salary - Target variable! 

* experience 

In [ ]:
q8 = pd.get_dummies(multiple.Experience)
q8.columns=list(map(lambda x: '8_'+x,q8.columns))
ndata = pd.concat([ndata,q8],axis=1)

* Q10 - ML 회사 사용? 

In [ ]:
multiple.columns[1:15]

Index(['gender', 'Age', 'country', 'degree', 'undergraduate_major', 'Role',
       'industry', 'Experience', 'value', 'Q10', 'Q11_Part_1', 'Q11_Part_2',
       'Q11_Part_3', 'Q11_Part_4'],
      dtype='object')

In [ ]:
ndata.shape

(14092, 77)

In [ ]:
# to dummies : 
q10 = pd.get_dummies(multiple.Q10)
q10.columns=list(map(lambda x: '10_'+x,q10.columns))
ndata = pd.concat([ndata,q10],axis=1)

In [ ]:
ndata.shape

(14092, 83)

* Q11

In [ ]:
def to_onehot(i):
    num = 'Q'+str(i)
    q = multiple[[t for t in multiple.columns if num in t]]
    q = q.drop([t for t in q.columns if 'OTHER_TEXT' in t],axis=1)  
    q=q.fillna(0)
    q_r = []
    for col in q.columns:
        if q[col].unique()[0]!=0:
            s = q[col].unique()[0]
            q_r.append(s)
        else:
            s = q[col].unique()[1]
            q_r.append(s)
        q[col]=q[col].replace(s,1)
    q.columns=q_r
    return q

In [ ]:
q11=to_onehot(11)
q11=q11.idxmax(axis=1)
q11.head()

0    Analyze and understand data to influence produ...
1                                                Other
2    Analyze and understand data to influence produ...
3    Build and/or run a machine learning service th...
4    None of these activities are an important part...
dtype: object

In [ ]:
q11=to_onehot(11)
q11.columns = ['11_'+str(col) for col in q11.columns]

In [ ]:
#Add None of these & Others - to others
q11.iloc[:,-1]=q11.iloc[:,-2]+q11.iloc[:,-1]
q11=q11.drop(q11.columns[-2],axis=1)

In [ ]:
ndata=pd.concat([ndata,q11],axis=1)

## 원래부터 One-Hot Encoding 된 것들 : Nan -> 0로 바꾸기. 

* Q12

In [ ]:
q12=multiple[[t for t in multiple.columns if 'Q12' in t]]
q12=q12.iloc[:,1:]
q12=q12.astype(int) 
q12[q12!=-1]=1
q12[q12==-1]=0
q12.head()

,Q12_Part_2_TEXT,Q12_Part_3_TEXT,Q12_Part_4_TEXT,Q12_Part_5_TEXT
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,1,0,0,0


In [ ]:
q12.columns=['12_'+ str(col) for col in q12.columns]
ndata=pd.concat([ndata,q12],axis=1)

* Q13

In [ ]:
def preprocess_multiple(data,i):
    s='Q'+str(i)
    if s+'_OTHER_TEXT' in data.columns:
        data=data.drop([s+'_OTHER_TEXT'],axis=1)
    df=to_onehot(i)
    df.columns=[str(i)+'_'+str(col) for col in df.columns]
    return df

In [ ]:
q13=preprocess_multiple(multiple, 13)
ndata=pd.concat([ndata,q13],axis=1)

In [ ]:
q13.head()

,13_Jupyter/IPython,13_RStudio,13_PyCharm,13_Visual Studio Code,13_nteract,13_Atom,13_MATLAB,13_Visual Studio,13_Notepad++,13_Sublime Text,13_Vim,13_IntelliJ,13_Spyder,13_None,13_Other
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0
4,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0


In [ ]:
ndata.shape

(14092, 108)

In [ ]:
ndata.iloc[1:5, 60:74]

,7_Online Business/Internet-based Sales,7_Online Service/Internet-based Services,7_Other,7_Retail/Sales,7_Shipping/Transportation,7_Student,8_0-1,8_1-2,8_10-15,8_15-20,8_2-3,8_20-25,8_25-30,8_3-4
1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,0,0,0,1


* Q14

In [ ]:
q14=preprocess_multiple(multiple, 14)
ndata=pd.concat([ndata,q14],axis=1)

* Q15 - Q16

In [ ]:
q15=preprocess_multiple(multiple, 15)
ndata=pd.concat([ndata,q15],axis=1)

In [ ]:
q16=preprocess_multiple(multiple, 16)
ndata=pd.concat([ndata,q16],axis=1)

In [ ]:
preprocess_needed = [19, 21]

In [ ]:
ndata2 = ndata.copy()

In [ ]:
for i in preprocess_needed : 
  q = preprocess_multiple(multiple, i)
  ndata2 = pd.concat([ndata2, q], axis = 1)

* Q 23-Q26 : get_dummies

In [ ]:
q23=pd.get_dummies(multiple.Q23)
q23.columns=['23_'+str(col) for col in q23.columns]
ndata2=pd.concat([ndata2,q23],axis=1)

In [ ]:
lst=['I have never written code and I do not want to learn','I have never written code but I want to learn','< 1 year','1-2 years','3-5 years',   '5-10 years',
       '10-20 years', '20-30 years','30-40 years','40+ years']

In [ ]:
q24=pd.get_dummies(multiple.Q24)
q24.columns=['24_'+str(col) for col in q24.columns]
q24=q24.reindex(columns=list(map(lambda x: '24_'+x,lst)))
ndata2=pd.concat([ndata2,q24],axis=1)

* Q25

In [ ]:
multiple.Q25.unique()

array(['< 1 year', '2-3 years', '1-2 years', '5-10 years', '3-4 years',
       'I have never studied machine learning but plan to learn in the future',
       'I have never studied machine learning and I do not plan to',
       '4-5 years', '20+ years', '10-15 years'], dtype=object)

In [ ]:
q25=pd.get_dummies(multiple.Q25)
q25.columns=['25_'+str(col) for col in q25.columns]
ndata2=pd.concat([ndata2,q25],axis=1)

* Q26 : confindence

In [ ]:
tmp = multiple.confidence.copy()
len(tmp[tmp.isna()])

0

In [ ]:
q26=pd.get_dummies(multiple.confidence)
q26.columns=['26_'+str(col) for col in q26.columns]
ndata2=pd.concat([ndata2,q26],axis=1)

In [ ]:
ndata2.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,2_40-44,2_45-49,2_50-54,2_55-59,2_60-69,2_70-79,2_80+,3_Africa,3_Asia,3_Europe,3_NorthAmerica,3_Oceania,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,4_No formal education past high school,4_Professional degree,4_Some college/university study without earning a bachelor’s degree,Business Analyst,Data Analyst,Data Engineer/DBA,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Environmental science or geology,5_Fine arts or performing arts,"5_Humanities (history, literature, philosophy, etc.)",...,21_D3,21_Plotly,21_Bokeh,21_Seaborn,21_Geoplotlib,21_Leaflet,21_Lattice,21_None,21_Other,23_0% of my time,23_1% to 25% of my time,23_100% of my time,23_25% to 49% of my time,23_50% to 74% of my time,23_75% to 99% of my time,24_I have never written code and I do not want to learn,24_I have never written code but I want to learn,24_< 1 year,24_1-2 years,24_3-5 years,24_5-10 years,24_10-20 years,24_20-30 years,24_30-40 years,24_40+ years,25_1-2 years,25_10-15 years,25_2-3 years,25_20+ years,25_3-4 years,25_4-5 years,25_5-10 years,25_< 1 year,25_I have never studied machine learning and I do not plan to,25_I have never studied machine learning but plan to learn in the future,26_Definitely not,26_Definitely yes,26_Maybe,26_Probably not,26_Probably yes
0,718,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1,731,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,1142,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,751,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,2028,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


* Q27-Q31, Q33, Q36, Q38, Q42, Q44, Q45, Q47, Q49, Q50

In [ ]:
preprocess_needed = [27, 28, 29, 30, 31, 36]

In [ ]:
multiple=multiple.drop(['Q38_Part_19'],axis=1)
multiple=multiple.drop(['Q38_Part_20'],axis=1)

In [ ]:
for i in preprocess_needed : 
  q = preprocess_multiple(multiple, i)
  ndata2 = pd.concat([ndata2, q], axis = 1)

In [ ]:
ndata2.iloc[1:5, 300:310]
ndata2.shape

(14092, 348)

In [ ]:
ndata3 = pd.DataFrame

In [ ]:
ndata3 = ndata2.copy()

In [ ]:
for i in [38,42,44,45,47,49,50] : 
  q = preprocess_multiple(multiple, i)
  ndata3 = pd.concat([ndata3, q], axis = 1)

In [ ]:
ndata3.iloc[1:4, 310:330]

,30_Oracle Exadata,30_Oracle Warehouse Builder,30_SAP IQ,30_Snowflake,30_Databricks,30_Azure SQL Data Warehouse,30_Azure HDInsight,30_Azure Stream Analytics,30_IBM InfoSphere DataStorage,30_IBM Cloud Analytics Engine,30_IBM Cloud Streaming Analytics,30_None,30_Other,31_Audio Data,31_Categorical Data,31_Genetic Data,31_Geospatial Data,31_Image Data,31_Numerical Data,31_Sensor Data
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


* Q32

In [ ]:
multiple=multiple.drop(['Q32_OTHER'],axis=1)

* Q34 : 일반적으로 데이터 사이언스 프로젝트 진행 시 각 업무별로 소요되는 시간 비중  =>  빼기. 
-



In [ ]:
len(multiple.Q34_Part_1[multiple.Q34_Part_1.isna()]) 

1785

In [ ]:
q34=multiple[[t for t in multiple.columns if 'Q34_' in t]]/100

In [ ]:
ndata3=pd.concat([ndata3,q34],axis=1)

In [ ]:
q34.head()

,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6
0,0.02,0.03,0.20,0.50,0.2,0.00
1,0.10,0.10,0.20,0.10,0.2,0.23
2,0.12,0.15,0.13,0.30,0.1,0.00
3,0.55,0.10,0.20,0.05,0.0,0.10
4,0.00,0.20,0.30,0.50,0.0,0.00


* Q35

In [ ]:
len(multiple.Q35_Part_1[multiple.Q35_Part_1.isna()]) #NA값 마찬가지로 제외하는게 좋을듯

1938

In [ ]:
q35=multiple[[t for t in multiple.columns if 'Q35_' in t]]/100
q35.columns=['35_'+str(col) for col in q35.columns]
ndata3=pd.concat([ndata3, q35],axis=1)

* Q40, Q41, Q43, Q46, Q48

In [ ]:
multiple.Q40.unique()

array(['Independent projects are equally important as academic achievements',
       'Independent projects are slightly more important than academic achievements',
       'Independent projects are slightly less important than academic achievements',
       'Independent projects are much less important than academic achievements',
       'Independent projects are much more important than academic achievements',
       'No opinion; I do not know', nan], dtype=object)

In [ ]:
len(multiple.Q40[multiple.Q40.isna()]) #NA값삭제

1983

In [ ]:
#Numeric encoding
encode={'No opinion; I do not know':0, 'Independent projects are much less important than academic achievements':1,
        'Independent projects are slightly less important than academic achievements':2,
        'Independent projects are equally important as academic achievements':3, 
        'Independent projects are slightly more important than academic achievements':4,
        'Independent projects are much more important than academic achievements':5}
tmp=multiple.Q40.copy()
tmp=tmp.replace(encode)

In [ ]:
q40=pd.get_dummies(multiple.Q40)
q40.columns=['40_'+str(col) for col in q40.columns]

In [ ]:
ndata3 = pd.concat([ndata3,q40],axis=1)

* Q41 => 제거할수도. or 3점 척도 -> /3으로 range 1 사이로 변형 

In [ ]:
##Q41
encode = {'Very important':3, 'Slightly important':2,
       'Not at all important':1, 'No opinion; I do not know':0}
tmp = multiple[[i for i in multiple.columns if 'Q41_' in i]]
tmp = tmp.replace(encode)
tmp = tmp.fillna(0)

tmp = tmp.astype(int)
tmp = tmp/3

In [ ]:
ndata3 = pd.concat([ndata3,tmp],axis=1)

In [ ]:
ndata3.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,2_40-44,2_45-49,2_50-54,2_55-59,2_60-69,2_70-79,2_80+,3_Africa,3_Asia,3_Europe,3_NorthAmerica,3_Oceania,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,4_No formal education past high school,4_Professional degree,4_Some college/university study without earning a bachelor’s degree,Business Analyst,Data Analyst,Data Engineer/DBA,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Environmental science or geology,5_Fine arts or performing arts,"5_Humanities (history, literature, philosophy, etc.)",...,49_Share both data and code on Github or a similar code-sharing repository,"49_Share data, code, and environment using a hosted service (Kaggle Kernels, Google Colaboratory, Amazon SageMaker, etc.)","49_Share data, code, and environment using containers (Docker, etc.)","49_Share code, data, and environment using virtual machines (VirtualBox, etc.)",49_Make sure the code is well documented,49_Make sure the code is human-readable,49_Define all random seeds,49_Define relative rather than absolute file paths,49_Include a text file describing all dependencies,49_None/I do not make my work easy for others to reproduce,49_Other,50_Too expensive,50_Too time-consuming,50_Requires too much technical knowledge,50_Afraid that others will use my work without giving proper credit,50_Not enough incentives to share my work,50_I had never considered making my work easier for others to reproduce,50_None of these reasons apply to me,50_Other,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,35_Q35_Part_1,35_Q35_Part_2,35_Q35_Part_3,35_Q35_Part_4,35_Q35_Part_5,35_Q35_Part_6,40_Independent projects are equally important as academic achievements,40_Independent projects are much less important than academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly less important than academic achievements,40_Independent projects are slightly more important than academic achievements,40_No opinion; I do not know,Q41_Part_1,Q41_Part_2,Q41_Part_3
0,718,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0.02,0.03,0.20,0.50,0.2,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000
1,731,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0.10,0.10,0.20,0.10,0.2,0.23,0.2,0.4,0.2,0.2,0.0,0.0,0,0,0,0,1,0,0.333333,0.666667,1.000000
2,1142,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0.12,0.15,0.13,0.30,0.1,0.00,0.4,0.2,0.1,0.0,0.3,0.0,0,0,0,1,0,0,1.000000,1.000000,1.000000
3,751,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0.55,0.10,0.20,0.05,0.0,0.10,0.7,0.3,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0.333333,1.000000,0.666667
4,2028,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0.00,0.20,0.30,0.50,0.0,0.00,0.0,0.5,0.0,0.4,0.1,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000


1 0 Recoding 

In [ ]:
##Q41
encode = {'Very important':3, 'Slightly important':2,
       'Not at all important':1, 'No opinion; I do not know':0}
q41 = multiple[[i for i in multiple.columns if 'Q41_' in i]]
q41 = q41.replace(encode)
q41 = q41.fillna(0)
q41=q41.astype(int) 
q41[q41!=3]=0
q41[q41==3]=1
q41.head()

,Q41_Part_1,Q41_Part_2,Q41_Part_3
0,1,1,1
1,0,0,1
2,1,1,1
3,0,1,0
4,1,1,1


* Q43 : drop - too many classes. 척도형 

In [ ]:
multiple.Q43.unique()

array(['0-10', '20-30', '0', '10-20', '30-40', nan, '60-70', '40-50',
       '90-100', '70-80', '50-60', '80-90'], dtype=object)

In [ ]:
len(multiple.Q43[multiple.Q43.isna()]) #NA값 삭제

3962

In [ ]:
#numeric encoding
encode = {'0-10':1, '20-30':3, '0':0, '10-20':2, '30-40':4, '60-70':7, '40-50':5,
       '90-100':10, '70-80':8, '50-60':6, '80-90':9}
tmp = multiple.Q43.copy()
tmp = tmp.replace(encode)

In [ ]:
q43=pd.get_dummies(multiple.Q43)
q43.columns=['43_'+str(col) for col in q43.columns]
ndata3=pd.concat([ndata3,q43],axis=1)

* Q46 - drop : too many classes 

In [ ]:
multiple.Q46.unique()

array(['10-20', '20-30', '0', '50-60', '0-10', '40-50', '90-100', nan,
       '30-40', '70-80', '80-90', '60-70'], dtype=object)

In [ ]:
#numeric encoding
encode = {'0-10':1, '20-30':3, '0':0, '10-20':2, '30-40':4, '60-70':7, '40-50':5,
       '90-100':10, '70-80':8, '50-60':6, '80-90':9}
tmp = multiple.Q46.copy()
tmp = tmp.replace(encode)

In [ ]:
q46=pd.get_dummies(multiple.Q46)
q46.columns=['46_'+str(col) for col in q46.columns]
ndata3=pd.concat([ndata3,q46],axis=1)

* Q48

In [ ]:
multiple.Q48.unique()

array(['I am confident that I can explain the outputs of most if not all ML models',
       'I am confident that I can understand and explain the outputs of many but not all ML models',
       'I view ML models as "black boxes" but I am confident that experts are able to explain model outputs',
       'I do not know; I have no opinion on the matter',
       'Yes, most ML models are "black boxes"', nan], dtype=object)

In [ ]:
#numeric encoding
encode = {'I am confident that I can explain the outputs of most if not all ML models':1,
       'Yes, most ML models are "black boxes"':4,
       'I am confident that I can understand and explain the outputs of many but not all ML models':2,
       'I view ML models as "black boxes" but I am confident that experts are able to explain model outputs':3,
       'I do not know; I have no opinion on the matter':0}
tmp = multiple.Q48.copy()
tmp = tmp.replace(encode)

In [ ]:
q48=pd.get_dummies(multiple.Q48)
q48.columns=['48_'+str(col) for col in q48.columns]
ndata3=pd.concat([ndata3,q48],axis=1)

In [ ]:
ndata3.shape

(14092, 469)

In [ ]:
ndata3.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,2_40-44,2_45-49,2_50-54,2_55-59,2_60-69,2_70-79,2_80+,3_Africa,3_Asia,3_Europe,3_NorthAmerica,3_Oceania,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,4_No formal education past high school,4_Professional degree,4_Some college/university study without earning a bachelor’s degree,Business Analyst,Data Analyst,Data Engineer/DBA,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Environmental science or geology,5_Fine arts or performing arts,"5_Humanities (history, literature, philosophy, etc.)",...,35_Q35_Part_3,35_Q35_Part_4,35_Q35_Part_5,35_Q35_Part_6,40_Independent projects are equally important as academic achievements,40_Independent projects are much less important than academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly less important than academic achievements,40_Independent projects are slightly more important than academic achievements,40_No opinion; I do not know,Q41_Part_1,Q41_Part_2,Q41_Part_3,43_0,43_0-10,43_10-20,43_20-30,43_30-40,43_40-50,43_50-60,43_60-70,43_70-80,43_80-90,43_90-100,46_0,46_0-10,46_10-20,46_20-30,46_30-40,46_40-50,46_50-60,46_60-70,46_70-80,46_80-90,46_90-100,48_I am confident that I can explain the outputs of most if not all ML models,48_I am confident that I can understand and explain the outputs of many but not all ML models,48_I do not know; I have no opinion on the matter,"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs","48_Yes, most ML models are ""black boxes"""
0,718,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0.0,1.0,0.0,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,731,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.2,0.2,0.0,0.0,0,0,0,0,1,0,0.333333,0.666667,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,1142,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0.1,0.0,0.3,0.0,0,0,0,1,0,0,1.000000,1.000000,1.000000,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,751,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0.333333,1.000000,0.666667,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,2028,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.0,0.4,0.1,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## Salary : 

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(multiple.value)
le.classes_

array(['0-10,000', '10-20,000', '100-125,000', '125-150,000',
       '150-200,000', '20-30,000', '200-250,000', '250-300,000',
       '30-40,000', '300-400,000', '40-50,000', '400-500,000',
       '50-60,000', '500,000+', '60-70,000', '70-80,000', '80-90,000',
       '90-100,000',
       'I do not wish to disclose my approximate yearly compensation'],
      dtype=object)

In [ ]:
salary = le.transform(multiple.value)
salary = pd.Series(salary)
salary = salary.rename("value")

In [ ]:
len(salary)

14092

In [ ]:
df_labeled = pd.concat([ndata3, salary], axis=1)

In [ ]:
df_labeled.shape

(14092, 470)

* 상위 20% 변수 생성 

In [ ]:
not_disclosed = multiple[multiple.value == 'I do not wish to disclose my approximate yearly compensation'].index
multiple2 = pd.DataFrame()
multiple2 = multiple.copy()
multiple2['value'] = multiple['value'].replace(['I do not wish to disclose my approximate yearly compensation'], '0-0')

In [ ]:
# Define target variable
compensation = multiple2.value.str.replace(',', '').str.replace('500000\+', '500-500000').str.split('-')
multiple2['yearly_compensation_numerical'] = compensation.apply(lambda x: (int(x[0]) * 1000 + int(x[1]))/ 2) / 1000 # it is calculated in thousand dollars

In [ ]:
top20flag = multiple2.yearly_compensation_numerical.quantile(0.8)
print(top20flag)

65.0


In [ ]:
clean_dataset = (multiple2.yearly_compensation_numerical > top20flag).reset_index().astype(int) 
clean_dataset.columns = ['index', 'top20']

In [ ]:
len(clean_dataset.loc[clean_dataset['top20']==1, :])

2513

In [ ]:
clean_dataset.shape

(14092, 2)

In [ ]:
ndata_top = pd.DataFrame()
ndata_top = ndata3.copy()

In [ ]:
# Join with treated questions
for key, value in ndata_top.items():
    value = value.reset_index(drop=True)
    clean_dataset = clean_dataset.join(value, how='left')

In [ ]:
clean_dataset.top20[1:20]

1     0
2     0
3     0
4     0
5     1
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    1
15    0
16    1
17    0
18    0
19    0
Name: top20, dtype: int64

In [ ]:
multiple2.value[1:20]

1        0-10,000
2             0-0
3       20-30,000
4             0-0
5     125-150,000
6       30-40,000
7       30-40,000
8       10-20,000
9       30-40,000
10      50-60,000
11      30-40,000
12            0-0
13       0-10,000
14      70-80,000
15       0-10,000
16      80-90,000
17      10-20,000
18      10-20,000
19            0-0
Name: value, dtype: object

In [ ]:
clean_dataset = clean_dataset.drop('index', axis=1)

In [ ]:
df_labeled.columns[399:401]

Index(['47_Other', '47_Visualising and interpreting predicted vs. actual'], dtype='object')

In [ ]:
clean_dataset.shape

(14092, 470)

* class 3개로 쪼개보기 

In [ ]:
#numeric encoding
encode = {'0-10,000':0, '10-20,000':0, '20-30,000':0, '30-40,000':0, '40-50,000':0, '50-60,000':0, '60-70,000':0,
       '70-80,000':1, '80-90,000':1, '90-100,000':1, '100-125,000':1, '125-150,000':1,'150-200,000':1, '200-250,000':1, 
                       '250-300,000':1,'300-400,000':1,'400-500,000':1, '500,000+':1,'I do not wish to disclose my approximate yearly compensation':2}

tmp = multiple.value.copy()
tmp = tmp.replace(encode)

In [ ]:
tmp.head()

0    0
1    0
2    2
3    0
4    2
Name: value, dtype: int64

In [ ]:
clean_dataset_new = pd.concat([ndata3, tmp], axis=1)

In [ ]:
clean_dataset_new.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,2_40-44,2_45-49,2_50-54,2_55-59,2_60-69,2_70-79,2_80+,3_Africa,3_Asia,3_Europe,3_NorthAmerica,3_Oceania,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,4_No formal education past high school,4_Professional degree,4_Some college/university study without earning a bachelor’s degree,Business Analyst,Data Analyst,Data Engineer/DBA,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Environmental science or geology,5_Fine arts or performing arts,"5_Humanities (history, literature, philosophy, etc.)",...,35_Q35_Part_4,35_Q35_Part_5,35_Q35_Part_6,40_Independent projects are equally important as academic achievements,40_Independent projects are much less important than academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly less important than academic achievements,40_Independent projects are slightly more important than academic achievements,40_No opinion; I do not know,Q41_Part_1,Q41_Part_2,Q41_Part_3,43_0,43_0-10,43_10-20,43_20-30,43_30-40,43_40-50,43_50-60,43_60-70,43_70-80,43_80-90,43_90-100,46_0,46_0-10,46_10-20,46_20-30,46_30-40,46_40-50,46_50-60,46_60-70,46_70-80,46_80-90,46_90-100,48_I am confident that I can explain the outputs of most if not all ML models,48_I am confident that I can understand and explain the outputs of many but not all ML models,48_I do not know; I have no opinion on the matter,"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs","48_Yes, most ML models are ""black boxes""",value
0,718,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,1.0,0.0,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,731,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.2,0.0,0.0,0,0,0,0,1,0,0.333333,0.666667,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1142,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.3,0.0,0,0,0,1,0,0,1.000000,1.000000,1.000000,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2
3,751,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,1,0,0,0,0,0,0.333333,1.000000,0.666667,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,2028,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0.4,0.1,0.0,1,0,0,0,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2


In [ ]:
[col for col in clean_dataset_new.columns if '47_' in col]
len(clean_dataset_new['47_Visualising and interpreting predicted vs. actual'][clean_dataset_new['47_Visualising and interpreting predicted vs. actual']==1])
clean_dataset_new = clean_dataset_new.drop(['47_Visualising and interpreting predicted vs. actual'],axis=1)

In [ ]:
colnames = clean_dataset_new.columns
colnames[390:410]

Index(['47_Dimensionality reduction techniques',
       '47_Attention mapping/saliency mapping',
       '47_Plot predicted vs. actual results', '47_Print out a decision tree',
       '47_Sensitivity analysis/perturbation importance', '47_LIME functions',
       '47_ELI5 functions', '47_SHAP functions',
       '47_None/I do not use these model explanation techniques', '47_Other',
       '49_Share code on Github or a similar code-sharing repository',
       '49_Share both data and code on Github or a similar code-sharing repository',
       '49_Share data, code, and environment using a hosted service (Kaggle Kernels, Google Colaboratory, Amazon SageMaker, etc.)',
       '49_Share data, code, and environment using containers (Docker, etc.)',
       '49_Share code, data, and environment using virtual machines (VirtualBox, etc.)',
       '49_Make sure the code is well documented',
       '49_Make sure the code is human-readable', '49_Define all random seeds',
       '49_Define relative rath

In [ ]:
percent_answer = clean_dataset_new.sum(axis=0) / clean_dataset_new.shape[0]

for row in percent_answer.iteritems():
  if row[1] < 0.05 :
    clean_dataset_new = clean_dataset_new.drop(row[0], axis=1)

* Dropping answers that have less than 5% of repondents - to avoid overfitting. 

In [ ]:
clean_dataset_new.shape

(14092, 261)

In [ ]:
clean_dataset_new.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,3_Asia,3_Europe,3_NorthAmerica,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,Data Analyst,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Mathematics or statistics,5_Physics or astronomy,7_Academics/Education,7_Accounting/Finance,7_Computers/Technology,7_Student,8_0-1,8_1-2,8_10-15,8_2-3,8_3-4,8_4-5,8_5-10,10_I do not know,10_No (we do not use ML methods),10_We are exploring ML methods (and may one day put a model into production),"10_We have well established ML methods (i.e., models in production for more than 2 years)",...,50_Too time-consuming,50_Requires too much technical knowledge,50_Afraid that others will use my work without giving proper credit,50_Not enough incentives to share my work,50_I had never considered making my work easier for others to reproduce,50_None of these reasons apply to me,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,35_Q35_Part_1,35_Q35_Part_2,35_Q35_Part_3,35_Q35_Part_4,35_Q35_Part_5,40_Independent projects are equally important as academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly more important than academic achievements,Q41_Part_1,Q41_Part_2,Q41_Part_3,43_0,43_0-10,43_10-20,43_20-30,46_0,46_0-10,46_10-20,46_20-30,46_30-40,46_40-50,46_90-100,48_I am confident that I can explain the outputs of most if not all ML models,48_I am confident that I can understand and explain the outputs of many but not all ML models,48_I do not know; I have no opinion on the matter,"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs","48_Yes, most ML models are ""black boxes""",value
0,718,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0.02,0.03,0.20,0.50,0.2,0.00,0.0,0.0,0.0,1.0,0.0,1,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,731,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0.10,0.10,0.20,0.10,0.2,0.23,0.2,0.4,0.2,0.2,0.0,0,0,1,0.333333,0.666667,1.000000,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
2,1142,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0.12,0.15,0.13,0.30,0.1,0.00,0.4,0.2,0.1,0.0,0.3,0,0,0,1.000000,1.000000,1.000000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
3,751,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0.55,0.10,0.20,0.05,0.0,0.10,0.7,0.3,0.0,0.0,0.0,1,0,0,0.333333,1.000000,0.666667,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2028,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0.00,0.20,0.30,0.50,0.0,0.00,0.0,0.5,0.0,0.4,0.1,1,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2


In [ ]:
for c in clean_dataset_new:
    print(c)
    clean_dataset_new[c]=pd.to_numeric(clean_dataset_new[c])

duration
1_Female
1_Male
2_18-21
2_22-24
2_25-29
2_30-34
2_35-39
3_Asia
3_Europe
3_NorthAmerica
3_SouthAmerica
4_Bachelor’s degree
4_Doctoral degree
4_Master’s degree
Data Analyst
Data Scientist
Software Engineer
Statistician/Research Scientist
Student
5_A business discipline (accounting, economics, finance, etc.)
5_Computer science (software engineering, etc.)
5_Engineering (non-computer focused)
5_Mathematics or statistics
5_Physics or astronomy
7_Academics/Education
7_Accounting/Finance
7_Computers/Technology
7_Student
8_0-1
8_1-2
8_10-15
8_2-3
8_3-4
8_4-5
8_5-10
10_I do not know
10_No (we do not use ML methods)
10_We are exploring ML methods (and may one day put a model into production)
10_We have well established ML methods (i.e., models in production for more than 2 years)
10_We recently started using ML methods (i.e., models in production for less than 2 years)
10_We use ML methods for generating insights (but do not put working models into production)
11_Analyze and understand 

In [ ]:
clean_dataset_new.shape

(14092, 261)

* multicollinearity 제거 

In [ ]:
# Create correlation matrix
correl = clean_dataset_new.corr().abs()

# Select upper triangle of correlation matrix
upper = correl.where(np.triu(np.ones(correl.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.5
to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]

# Drop features 
#clean_dataset_dropped = clean_dataset.drop(to_drop, axis=1)

#shape = clean_dataset_dropped.shape
#print('After dropping highly correlated features, our has {} records and {} features'.format(shape[0], shape[1]))
#print('Dropped features: ', to_drop)

In [ ]:
to_drop

['1_Male', '7_Student']

In [ ]:
clean_dataset_new = clean_dataset_new.drop('7_Student', axis=1)

In [ ]:
upper

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,3_Asia,3_Europe,3_NorthAmerica,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,Data Analyst,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Mathematics or statistics,5_Physics or astronomy,7_Academics/Education,7_Accounting/Finance,7_Computers/Technology,7_Student,8_0-1,8_1-2,8_10-15,8_2-3,8_3-4,8_4-5,8_5-10,10_I do not know,10_No (we do not use ML methods),10_We are exploring ML methods (and may one day put a model into production),"10_We have well established ML methods (i.e., models in production for more than 2 years)",...,50_Too time-consuming,50_Requires too much technical knowledge,50_Afraid that others will use my work without giving proper credit,50_Not enough incentives to share my work,50_I had never considered making my work easier for others to reproduce,50_None of these reasons apply to me,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,35_Q35_Part_1,35_Q35_Part_2,35_Q35_Part_3,35_Q35_Part_4,35_Q35_Part_5,40_Independent projects are equally important as academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly more important than academic achievements,Q41_Part_1,Q41_Part_2,Q41_Part_3,43_0,43_0-10,43_10-20,43_20-30,46_0,46_0-10,46_10-20,46_20-30,46_30-40,46_40-50,46_90-100,48_I am confident that I can explain the outputs of most if not all ML models,48_I am confident that I can understand and explain the outputs of many but not all ML models,48_I do not know; I have no opinion on the matter,"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs","48_Yes, most ML models are ""black boxes""",value
duration,NaN,0.002181,0.004510,0.014024,0.004571,0.003576,0.003276,0.002181,0.001417,0.003292,0.011882,0.015800,0.007953,0.003829,0.002902,0.006872,0.015069,0.005951,0.011158,0.011168,0.012755,0.008336,0.000113,0.002631,0.001087,0.000253,0.008715,0.008932,0.012350,0.002971,0.011757,0.000091,0.009251,0.010280,0.003879,0.000491,0.005531,0.012293,0.016329,0.011062,...,0.009588,0.001152,0.011129,0.011524,0.002483,0.017551,0.018410,0.000942,0.004418,0.005582,0.002430,0.010023,0.004529,0.016150,0.002232,0.005856,0.018087,0.009469,0.016552,0.015926,0.007121,0.012164,0.015599,0.002703,0.010778,0.012383,0.004802,0.012898,0.015365,0.015058,0.000277,0.004172,0.006844,0.001741,0.002522,0.007489,0.004547,0.006173,0.018541,0.021850
1_Female,NaN,NaN,0.956512,0.017259,0.032846,0.026084,0.004550,0.009557,0.053890,0.018651,0.098184,0.042830,0.055014,0.017447,0.055218,0.037445,0.007275,0.071005,0.008746,0.045080,0.005107,0.038250,0.050472,0.054203,0.035786,0.015181,0.023546,0.021840,0.042084,0.045910,0.009732,0.025027,0.005184,0.007829,0.016175,0.032281,0.053974,0.007864,0.011947,0.028743,...,0.046467,0.006709,0.004008,0.028927,0.002342,0.002088,0.037275,0.004636,0.052745,0.000855,0.011740,0.040264,0.076579,0.017563,0.007644,0.134752,0.095938,0.029137,0.024925,0.009504,0.011424,0.016247,0.016205,0.014265,0.004168,0.006118,0.005849,0.012068,0.005245,0.019731,0.010129,0.012628,0.007703,0.003435,0.008833,0.023135,0.028163,0.000281,0.022353,0.037043
1_Male,NaN,NaN,NaN,0.020979,0.028160,0.024163,0.000306,0.008423,0.060873,0.018382,0.099463,0.044947,0.051989,0.023280,0.048391,0.033420,0.006118,0.071761,0.008163,0.044578,0.001155,0.038217,0.049520,0.051477,0.033505,0.020116,0.023175,0.022901,0.039857,0.038749,0.008811,0.023599,0.002604,0.007000,0.016139,0.027300,0.049918,0.004788,0.012149,0.025106,...,0.042846,0.011362,0.004385,0.028259,0.005132,0.005416,0.033402,0.003406,0.044632,0.001110,0.014530,0.039244,0.070669,0.023695,0.006131,0.134816,0.093849,0.028415,0.028617,0.007202,0.008233,0.010390,0.020211,0.014892,0.000378,0.003717,0.004868,0.0

* Q43, Q46, Q41 제거 근거. 

In [ ]:
# Finding NANs
df = clean_dataset_new.isnull().sum().to_frame()
print('We found {} NaNs on the dataset after treatment'.format(df[df[0] > 0].shape[0]))
## 질문을 날리는게 더 낫지않을까? 유의하게 안나오면 질문 날리고 진행해봅시다. 

We found 11 NaNs on the dataset after treatment


In [ ]:
df

,0
duration,0
1_Female,0
1_Male,0
2_18-21,0
2_22-24,0
...,...
48_I am confident that I can understand and explain the outputs of many but not all ML models,0
48_I do not know; I have no opinion on the matter,0
"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs",0
"48_Yes, most ML models are ""black boxes""",0


In [ ]:
for c in clean_dataset_new:
    if len(clean_dataset_new[clean_dataset_new[c].isna()])>0:
        clean_dataset_new=clean_dataset_new[~clean_dataset_new[c].isna()]

In [ ]:
clean_dataset_new.shape

(12154, 261)

In [ ]:
clean_dataset_new.iloc[:,100:].sum(axis=1)

0        33.950000
1        27.930000
2        50.800000
3        38.000000
4        39.000000
           ...    
14078    24.000000
14083    28.283333
14084    45.800000
14087    29.700000
14088    13.000000
Length: 12154, dtype: float64

In [ ]:
clean_dataset_new.head()

,duration,1_Female,1_Male,2_18-21,2_22-24,2_25-29,2_30-34,2_35-39,3_Asia,3_Europe,3_NorthAmerica,3_SouthAmerica,4_Bachelor’s degree,4_Doctoral degree,4_Master’s degree,Data Analyst,Data Scientist,Software Engineer,Statistician/Research Scientist,Student,"5_A business discipline (accounting, economics, finance, etc.)","5_Computer science (software engineering, etc.)",5_Engineering (non-computer focused),5_Mathematics or statistics,5_Physics or astronomy,7_Academics/Education,7_Accounting/Finance,7_Computers/Technology,7_Student,8_0-1,8_1-2,8_10-15,8_2-3,8_3-4,8_4-5,8_5-10,10_I do not know,10_No (we do not use ML methods),10_We are exploring ML methods (and may one day put a model into production),"10_We have well established ML methods (i.e., models in production for more than 2 years)",...,50_Too time-consuming,50_Requires too much technical knowledge,50_Afraid that others will use my work without giving proper credit,50_Not enough incentives to share my work,50_I had never considered making my work easier for others to reproduce,50_None of these reasons apply to me,Q34_Part_1,Q34_Part_2,Q34_Part_3,Q34_Part_4,Q34_Part_5,Q34_Part_6,35_Q35_Part_1,35_Q35_Part_2,35_Q35_Part_3,35_Q35_Part_4,35_Q35_Part_5,40_Independent projects are equally important as academic achievements,40_Independent projects are much more important than academic achievements,40_Independent projects are slightly more important than academic achievements,Q41_Part_1,Q41_Part_2,Q41_Part_3,43_0,43_0-10,43_10-20,43_20-30,46_0,46_0-10,46_10-20,46_20-30,46_30-40,46_40-50,46_90-100,48_I am confident that I can explain the outputs of most if not all ML models,48_I am confident that I can understand and explain the outputs of many but not all ML models,48_I do not know; I have no opinion on the matter,"48_I view ML models as ""black boxes"" but I am confident that experts are able to explain model outputs","48_Yes, most ML models are ""black boxes""",value
0,718,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0.02,0.03,0.20,0.50,0.2,0.00,0.0,0.0,0.0,1.0,0.0,1,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,731,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0.10,0.10,0.20,0.10,0.2,0.23,0.2,0.4,0.2,0.2,0.0,0,0,1,0.333333,0.666667,1.000000,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
2,1142,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0.12,0.15,0.13,0.30,0.1,0.00,0.4,0.2,0.1,0.0,0.3,0,0,0,1.000000,1.000000,1.000000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,2
3,751,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0.55,0.10,0.20,0.05,0.0,0.10,0.7,0.3,0.0,0.0,0.0,1,0,0,0.333333,1.000000,0.666667,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2028,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0.00,0.20,0.30,0.50,0.0,0.00,0.0,0.5,0.0,0.4,0.1,1,0,0,1.000000,1.000000,1.000000,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2


In [ ]:
clean_dataset_new.iloc[:, -1]

0        0
1        0
2        2
3        0
4        2
        ..
14078    0
14083    0
14084    1
14087    2
14088    0
Name: value, Length: 12154, dtype: int64

## Modeling 

* Random Forest : 

In [ ]:
from sklearn.model_selection import train_test_split

x = clean_dataset_new.iloc[:,1:-1]
y = clean_dataset_new.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1234)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(n_estimators=100, random_state=10)
scores = cross_val_score(rf, x_train, y_train, cv=10, scoring='accuracy')
print(round(scores.mean(),3))

0.657


In [ ]:
y.value_counts()

0    7018
2    2872
1    2264
Name: value, dtype: int64

In [ ]:
from sklearn.feature_selection import RFECV
## gridsearch 
rf = RandomForestClassifier(random_state=10, n_estimators=100)
selector = RFECV(rf, step = 1, cv=10, scoring = 'accuracy')

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm

rforest = RandomForestClassifier(n_estimators=100, random_state=1234)
#lreg = LogisticRegression(solver='liblinear', random_state=1234)
#svm = svm.SVC(kernel='linear', random_state=1234)

# Fit the models
rforest.fit(x_train, y_train)
#lreg.fit(x_train, y_train)
#svm.fit(x_train, y_train)

ypred = rforest.predict(x_train)
accuracy = accuracy_score(y_train, ypred)
confusion = confusion_matrix(y_train, ypred)

rf_scores = cross_val_score(rforest, x_train, y_train, cv=10, scoring='accuracy')
print(round(rf_scores.mean(),3))
print(accuracy)
print(confusion)

## Train - 완벽하게 분류됨.(Accuracy 기준), 하지만 CV score를 보면 0.657로 상당히 낮은 점수를 보여준다, 

0.657
1.0
[[4953    0    0]
 [   0 1566    0]
 [   0    0 1988]]


* Variable importance Plot 

In [ ]:
index_clean = clean_dataset_new.columns[1:259]
index_clean

Index(['1_Female', '1_Male', '2_18-21', '2_22-24', '2_25-29', '2_30-34',
       '2_35-39', '3_Asia', '3_Europe', '3_NorthAmerica',
       ...
       '46_10-20', '46_20-30', '46_30-40', '46_40-50', '46_90-100',
       '48_I am confident that I can explain the outputs of most if not all ML models',
       '48_I am confident that I can understand and explain the outputs of many but not all ML models',
       '48_I do not know; I have no opinion on the matter',
       '48_I view ML models as "black boxes" but I am confident that experts are able to explain model outputs',
       '48_Yes, most ML models are "black boxes"'],
      dtype='object', length=258)

In [ ]:
feature_imp = pd.Series(rforest.feature_importances_, index=index_clean).sort_values(ascending = False)
feature_imp[0:20]

3_NorthAmerica                        0.042415
Student                               0.019219
Q34_Part_4                            0.015646
35_Q35_Part_1                         0.015185
Q34_Part_2                            0.014926
Q34_Part_1                            0.014454
35_Q35_Part_2                         0.014256
35_Q35_Part_4                         0.013596
35_Q35_Part_3                         0.013221
Q34_Part_6                            0.013108
Q34_Part_3                            0.012873
Q34_Part_5                            0.012791
35_Q35_Part_5                         0.010243
10_I do not know                      0.009933
3_Asia                                0.008893
Data Scientist                        0.006842
Q41_Part_1                            0.006598
27_AWS Elastic Compute Cloud (EC2)    0.006558
2_22-24                               0.006011
3_Europe                              0.005849
dtype: float64

*북미가 역시 가장 높게 나타났다. 
* 그 뒤 = student
* Q34 Q35는 꾸준히 높게 나타나는 요인들.
* 1그 뒤 대륙으로는 Asia
* DS가 역시 유의한 영향을 가지고 있는 것으로 보임. 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

model_ovr = OneVsRestClassifier(LogisticRegression()).fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
